# `GiRaFFE_HO` C code library: Boundary conditions

### Author: Patrick Nelson

This writes and documents the C code that `GiRaFFE_HO` uses to apply boundary conditions to the GRFFE quantities. 

**Status:** These algorithms are under active development, and it is unclear which routine documented here is the most appropriate to use, and whether or not the implementation is entirely correct. 